In [2]:
income_statements = {}
with concurrent.futures.ProcessPoolExecutor() as executor:
    for ticker in stock_list:
        balance_sheets[ticker] = executor.submit(si.get_balance_sheet,ticker)
#         print(ticker)
print('done')

for ticker in stock_list:
    balance_sheets[ticker] = balance_sheets[ticker].result()
#     print(ticker + '2')
    
print('done2')

done
done2


In [3]:
recent_sheets = {ticker : sheet.iloc[:,:1] for ticker,sheet in balance_sheets.items()}
for ticker in recent_sheets.keys():
    recent_sheets[ticker].columns = ['Recent']
    
combined_sheets = pd.concat(recent_sheets)

combined_sheets = combined_sheets.reset_index()
combined_sheets.columns = ['Ticker', 'Breakdown', 'Recent']

In [4]:
income_statements = {}
with concurrent.futures.ProcessPoolExecutor() as executor:
    for ticker in stock_list:
        income_statements[ticker] = executor.submit(si.get_income_statement,ticker)
#         print(ticker)
print('done')

for ticker in stock_list:
    income_statements[ticker] = income_statements[ticker].result()
#     print(ticker + '2')
    
print('done2')

done
done2


In [5]:
recent_income_statements = {ticker : sheet.iloc[:,:1] for ticker,sheet in income_statements.items()}
for ticker in recent_income_statements.keys():
    recent_income_statements[ticker].columns = ['Recent']

combined_income = pd.concat(recent_income_statements)
combined_income = combined_income.reset_index()
combined_income.columns = ['Ticker', 'Breakdown', 'Recent']

In [7]:
cash_flow_statements = {}
with concurrent.futures.ProcessPoolExecutor() as executor:
    for ticker in stock_list:
        cash_flow_statements[ticker] = executor.submit(si.get_cash_flow,ticker)
#         print(ticker)
print('done')

for ticker in stock_list:
    cash_flow_statements[ticker] = cash_flow_statements[ticker].result()
#     print(ticker + '2')
    
print('done2')

done
done2


In [8]:
recent_cash_flow_statements = {ticker : sheet.iloc[:,:1] for ticker,sheet in income_statements.items()}
for ticker in recent_cash_flow_statements.keys():
    recent_cash_flow_statements[ticker].columns = ['Recent']

combined_cash_flow = pd.concat(recent_cash_flow_statements)
combined_cash_flow = combined_cash_flow.reset_index()
combined_cash_flow.columns = ['Ticker', 'Breakdown', 'Recent']

In [9]:
frames = [combined_sheets, combined_income, combined_cash_flow]
combined_df = pd.concat(frames)
combined_df = combined_df.reset_index()

In [11]:
pivot_df = combined_df.pivot_table(index='Ticker', columns='Breakdown', values='Recent', aggfunc='sum')

In [12]:
pivot_df.query("Ticker == 'AMZN'")

Breakdown,accountsPayable,capitalSurplus,cash,commonStock,costOfRevenue,deferredLongTermAssetCharges,deferredLongTermLiab,discontinuedOperations,ebit,effectOfAccountingCharges,...,shortTermInvestments,totalAssets,totalCurrentAssets,totalCurrentLiabilities,totalLiab,totalOperatingExpenses,totalOtherIncomeExpenseNet,totalRevenue,totalStockholderEquity,treasuryStock
Ticker,,,,,,,,,,,,,,,,,,,,,
AMZN,7.866400e+10,5.553800e+10,3.622000e+10,5000000.0,5.446880e+11,NaN,NaN,0.0,4.975800e+10,0.0,...,5.982900e+10,4.205490e+11,1.615800e+11,1.422660e+11,2.823040e+11,8.898860e+11,2.655200e+10,9.396440e+11,1.382450e+11,-3.213000e+09


In [13]:
pivot_df.query("ebit >= 0")

Breakdown,accountsPayable,capitalSurplus,cash,commonStock,costOfRevenue,deferredLongTermAssetCharges,deferredLongTermLiab,discontinuedOperations,ebit,effectOfAccountingCharges,...,shortTermInvestments,totalAssets,totalCurrentAssets,totalCurrentLiabilities,totalLiab,totalOperatingExpenses,totalOtherIncomeExpenseNet,totalRevenue,totalStockholderEquity,treasuryStock
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,5.476300e+10,NaN,3.494000e+10,5.736500e+10,4.259620e+11,NaN,NaN,0.000000e+00,2.178980e+11,0.0,...,2.769900e+10,3.510020e+11,1.348360e+11,1.254810e+11,2.879120e+11,5.137360e+11,5.160000e+08,7.316340e+11,6.309000e+10,1.630000e+08
AMGN,1.366000e+09,NaN,7.989000e+09,3.209600e+10,1.290800e+10,NaN,NaN,0.000000e+00,1.867600e+10,0.0,...,4.800000e+07,6.116500e+10,1.938500e+10,1.218400e+10,5.446500e+10,3.328200e+10,-5.274000e+09,5.195800e+10,6.700000e+09,-7.960000e+08
AMZN,7.866400e+10,5.553800e+10,3.622000e+10,5.000000e+06,5.446880e+11,NaN,NaN,0.000000e+00,4.975800e+10,0.0,...,5.982900e+10,4.205490e+11,1.615800e+11,1.422660e+11,2.823040e+11,8.898860e+11,2.655200e+10,9.396440e+11,1.382450e+11,-3.213000e+09
AXP,9.495600e+10,1.149500e+10,2.104000e+10,1.530000e+08,2.600000e+10,2.385000e+09,NaN,0.000000e+00,0.000000e+00,0.0,...,4.630000e+08,1.885480e+11,1.640180e+11,1.058250e+11,1.663710e+11,6.726000e+10,1.040000e+09,8.759800e+10,2.217700e+10,-2.945000e+09
CAT,8.154000e+09,NaN,8.428000e+09,6.398000e+09,7.556400e+10,1.669000e+09,NaN,0.000000e+00,1.654800e+10,0.0,...,9.640000e+08,8.279300e+10,4.345500e+10,2.984700e+10,6.627700e+10,8.539400e+10,-7.800000e+07,1.019420e+11,1.648400e+10,-2.919600e+10
CRM,2.956000e+09,5.091900e+10,5.464000e+09,1.000000e+06,1.405200e+10,2.544000e+09,2.421000e+09,0.000000e+00,1.204000e+09,0.0,...,5.073000e+09,9.520900e+10,2.285000e+10,2.178800e+10,3.707800e+10,5.178000e+10,1.860000e+09,5.298400e+10,5.813100e+10,-1.660000e+08
CSCO,2.513000e+09,NaN,9.175000e+09,4.234600e+10,3.584800e+10,4.360000e+09,NaN,0.000000e+00,2.753000e+10,0.0,...,1.534300e+10,9.749700e+10,3.911200e+10,2.625700e+10,5.622200e+10,7.210600e+10,-1.006000e+09,9.963600e+10,4.127500e+10,-4.170000e+08
CVX,1.645400e+10,1.728200e+10,5.640000e+09,1.832000e+09,1.787440e+11,5.659000e+09,2.235000e+09,0.000000e+00,3.172000e+10,0.0,...,3.500000e+07,2.395350e+11,3.373800e+10,2.679100e+10,9.959500e+10,2.794920e+11,1.155800e+10,3.112120e+11,1.390670e+11,-4.559300e+10
DIS,1.567900e+10,NaN,1.595900e+10,5.547100e+10,9.026200e+10,NaN,NaN,-5.800000e+07,6.984000e+09,0.0,...,NaN,2.036090e+11,3.365700e+10,3.107700e+10,1.013850e+11,1.278520e+11,-1.862000e+09,1.348360e+11,8.855300e+10,-7.347000e+09
